In [ ]:
#%%capture
#!pip install watson-machine-learning-client --user
#!pip uninstall -y watson-machine-learning-client
#!pip uninstall -y watson-machine-learning-client-V4
#!pip install -i https://test.pypi.org/simple/ watson-machine-learning-client-V4 --user
#!pip install --upgrade watson-machine-learning-client-V4 | tail -n 1
#!pip install watson-machine-learning-client-V4 --user
#!pip uninstall -y ibm-watson-machine-learning
!pip install ibm-cos-sdk==2.6.0 --user
!pip install ibm-watson-machine-learning --user
!pip install --upgrade ibm_watson_openscale --no-cache | tail -n 1
!pip install pyspark==2.3.3 --user

In [ ]:
# Enter credentials
CLOUD_API_KEY = "<>"
COS_RESOURCE_INSTANCE_ID = "<>"
WML_CRN = "<>"

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_2fd136904c704241b032b6a83f5d5769 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='<>',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_2fd136904c704241b032b6a83f5d5769.get_object(Bucket='demoproject-donotdelete-pr-mnpdjkegrmzbji',Key='german_credit_data_biased_training.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

pd_data = pd.read_csv(body)
pd_data.head()


In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = pyspark.SparkContext.getOrCreate()
sql_sc = SQLContext(sc)
df_data = sql_sc.createDataFrame(pd_data)

In [ ]:
#df_data.show()

In [ ]:
df_data.printSchema()

In [ ]:
print("Number of records: " + str(df_data.count()))

In [ ]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "German Credit Risk Model-V4"
DEPLOYMENT_NAME = "German Credit Risk Deployment-V4"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model

si_CheckingStatus = StringIndexer(inputCol = 'CheckingStatus', outputCol = 'CheckingStatus_IX')
si_CreditHistory = StringIndexer(inputCol = 'CreditHistory', outputCol = 'CreditHistory_IX')
si_LoanPurpose = StringIndexer(inputCol = 'LoanPurpose', outputCol = 'LoanPurpose_IX')
si_ExistingSavings = StringIndexer(inputCol = 'ExistingSavings', outputCol = 'ExistingSavings_IX')
si_EmploymentDuration = StringIndexer(inputCol = 'EmploymentDuration', outputCol = 'EmploymentDuration_IX')
si_Sex = StringIndexer(inputCol = 'Sex', outputCol = 'Sex_IX')
si_OthersOnLoan = StringIndexer(inputCol = 'OthersOnLoan', outputCol = 'OthersOnLoan_IX')
si_OwnsProperty = StringIndexer(inputCol = 'OwnsProperty', outputCol = 'OwnsProperty_IX')
si_InstallmentPlans = StringIndexer(inputCol = 'InstallmentPlans', outputCol = 'InstallmentPlans_IX')
si_Housing = StringIndexer(inputCol = 'Housing', outputCol = 'Housing_IX')
si_Job = StringIndexer(inputCol = 'Job', outputCol = 'Job_IX')
si_Telephone = StringIndexer(inputCol = 'Telephone', outputCol = 'Telephone_IX')
si_ForeignWorker = StringIndexer(inputCol = 'ForeignWorker', outputCol = 'ForeignWorker_IX')

In [ ]:
si_Label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [ ]:
va_features = VectorAssembler(inputCols=["CheckingStatus_IX", "CreditHistory_IX", "LoanPurpose_IX", "ExistingSavings_IX", "EmploymentDuration_IX", "Sex_IX", \
                                         "OthersOnLoan_IX", "OwnsProperty_IX", "InstallmentPlans_IX", "Housing_IX", "Job_IX", "Telephone_IX", "ForeignWorker_IX", \
                                         "LoanDuration", "LoanAmount", "InstallmentPercent", "CurrentResidenceDuration", "LoanDuration", "Age", "ExistingCreditsCount", \
                                         "Dependents"], outputCol="features")

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_CheckingStatus, si_CreditHistory, si_EmploymentDuration, si_ExistingSavings, si_ForeignWorker, si_Housing, si_InstallmentPlans, si_Job, si_LoanPurpose, si_OthersOnLoan,\
                               si_OwnsProperty, si_Sex, si_Telephone, si_Label, va_features, classifier, label_converter])
model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

In [ ]:
wml_credentials={
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": CLOUD_API_KEY,
}

In [ ]:
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)
client.version

In [ ]:
metadata = {
           client.spaces.ConfigurationMetaNames.NAME: 'WML-V4-Prod',
           client.spaces.ConfigurationMetaNames.DESCRIPTION: 'Space for V4 models deployments in WML YPPROD',
           client.spaces.ConfigurationMetaNames.STORAGE: {
               "resource_crn": COS_RESOURCE_INSTANCE_ID,
               "type": "bmcos_object_storage"
           },
           client.spaces.ConfigurationMetaNames.COMPUTE: {
               "name": "avmwj2-wml-v4",
               "crn": WML_CRN,
               "type": "machine_learning"
          }
         }

details = client.spaces.store(meta_props=metadata)
space_id = details['metadata'].get('id')
print(space_id)

In [ ]:
client.spaces.get_details(space_id)

In [ ]:
client.set.default_space(space_id)

In [ ]:
software_spec_uid = client.software_specifications.get_id_by_name("spark-mllib_2.4")
print(software_spec_uid)

In [ ]:
model_props = {
    client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    client.repository.ModelMetaNames.SPACE_UID: space_id,
    client.repository.ModelMetaNames.TYPE: 'mllib_2.4',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

In [ ]:
published_model_details = client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
model_uid = client.repository.get_model_uid(published_model_details)
print("Model UID:" + model_uid)

In [ ]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [ ]:
deployment_details = client.deployments.create(artifact_uid=model_uid, meta_props=meta_props)

In [ ]:
scoring_url = deployment_details["entity"]["status"]["online_url"]["url"]
print("Scoring URL:" + scoring_url)

In [ ]:
deployment_uid = client.deployments.get_uid(deployment_details)
print("Deployment UID:" + deployment_uid)

In [ ]:
# WML Scoring
scoring_payload = {
    "input_data": [{
        "values": [
            ["no_checking", 30, "prior_payments_delayed", "appliances", 4562, "unknown", "4_to_7", 3, "male", "none", 4, "unknown", 46, "bank", "rent", 2, "unskilled", 2, "yes", "yes"],
            ["0_to_200", 33, "prior_payments_delayed", "radio_tv", 5252, "100_to_500", "1_to_4", 4, "female", "none", 2, "savings_insurance", 34, "none", "free", 2, "skilled", 1, "none", "yes"],
            ["0_to_200", 5, "all_credits_paid_back", "car_new", 250, "500_to_1000", "less_1", 2, "female", "none", 3, "real_estate", 20, "none", "rent", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 43, "outstanding_credit", "education", 6888, "unknown", "greater_7", 5, "male", "co-applicant", 5, "unknown", 51, "none", "free", 2, "skilled", 2, "yes", "no"],
            ["no_checking", 14, "outstanding_credit", "car_new", 1679, "500_to_1000", "1_to_4", 3, "female", "none", 3, "savings_insurance", 48, "stores", "own", 2, "skilled", 1, "yes", "yes"],
            ["no_checking", 29, "prior_payments_delayed", "appliances", 5605, "unknown", "greater_7", 4, "male", "none", 5, "car_other", 42, "none", "own", 3, "management_self-employed", 2, "none", "yes"],
            ["0_to_200", 6, "all_credits_paid_back", "car_new", 734, "less_100", "less_1", 2, "female", "none", 3, "real_estate", 19, "none", "own", 1, "management_self-employed", 1, "none", "yes"],
            ["less_0", 9, "all_credits_paid_back", "car_new", 1709, "less_100", "less_1", 1, "male", "none", 1, "car_other", 37, "stores", "rent", 1, "skilled", 1, "none", "yes"],
            ["less_0", 8, "all_credits_paid_back", "car_new", 250, "less_100", "less_1", 1, "male", "none", 1, "savings_insurance", 29, "stores", "rent", 1, "skilled", 1, "none", "yes"],
            ["greater_200", 6, "credits_paid_to_date", "car_new", 6193, "less_100", "4_to_7", 4, "male", "none", 2, "savings_insurance", 27, "none", "own", 1, "management_self-employed", 1, "none", "yes"],
            ["less_0", 11, "prior_payments_delayed", "car_used", 728, "less_100", "4_to_7", 2, "male", "none", 3, "car_other", 30, "stores", "own", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 19, "credits_paid_to_date", "retraining", 3258, "100_to_500", "less_1", 3, "male", "none", 4, "savings_insurance", 19, "none", "rent", 1, "management_self-employed", 1, "none", "yes"],
            ["less_0", 13, "all_credits_paid_back", "car_new", 656, "less_100", "1_to_4", 2, "female", "none", 1, "real_estate", 24, "stores", "own", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 33, "outstanding_credit", "furniture", 4527, "greater_1000", "greater_7", 5, "male", "none", 5, "savings_insurance", 44, "none", "free", 2, "skilled", 1, "yes", "yes"],
            ["less_0", 9, "all_credits_paid_back", "car_new", 250, "less_100", "unemployed", 1, "male", "none", 1, "real_estate", 19, "bank", "rent", 1, "unemployed", 1, "none", "yes"],
            ["greater_200", 14, "prior_payments_delayed", "car_new", 4626, "greater_1000", "1_to_4", 3, "male", "none", 4, "savings_insurance", 37, "none", "own", 1, "skilled", 1, "yes", "yes"],
            ["no_checking", 41, "prior_payments_delayed", "appliances", 6098, "unknown", "4_to_7", 5, "male", "none", 5, "unknown", 45, "none", "own", 2, "skilled", 2, "yes", "yes"],
            ["0_to_200", 14, "prior_payments_delayed", "car_used", 3147, "less_100", "1_to_4", 2, "male", "none", 3, "savings_insurance", 53, "none", "free", 1, "skilled", 1, "yes", "yes"],
            ["no_checking", 27, "prior_payments_delayed", "appliances", 4959, "100_to_500", "1_to_4", 3, "female", "none", 4, "savings_insurance", 41, "bank", "own", 2, "unemployed", 2, "yes", "yes"],
            ["no_checking", 23, "prior_payments_delayed", "car_used", 1304, "500_to_1000", "greater_7", 4, "male", "none", 4, "real_estate", 43, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["less_0", 10, "credits_paid_to_date", "car_used", 4263, "less_100", "1_to_4", 3, "male", "none", 3, "savings_insurance", 26, "none", "own", 2, "skilled", 1, "yes", "yes"],
            ["no_checking", 20, "credits_paid_to_date", "radio_tv", 4259, "100_to_500", "4_to_7", 3, "female", "none", 4, "savings_insurance", 36, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["less_0", 17, "credits_paid_to_date", "repairs", 958, "less_100", "less_1", 2, "male", "none", 1, "savings_insurance", 23, "bank", "rent", 1, "unskilled", 2, "none", "yes"],
            ["less_0", 4, "credits_paid_to_date", "car_new", 250, "100_to_500", "less_1", 2, "female", "none", 2, "real_estate", 19, "none", "rent", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 29, "prior_payments_delayed", "furniture", 6619, "100_to_500", "4_to_7", 3, "male", "none", 3, "unknown", 40, "stores", "rent", 1, "unskilled", 1, "yes", "yes"],
            ["0_to_200", 15, "credits_paid_to_date", "vacation", 676, "less_100", "4_to_7", 2, "female", "none", 1, "savings_insurance", 31, "bank", "rent", 1, "unskilled", 1, "none", "yes"],
            ["no_checking", 38, "credits_paid_to_date", "appliances", 5679, "unknown", "greater_7", 5, "male", "none", 5, "savings_insurance", 42, "none", "own", 3, "skilled", 2, "none", "yes"],
            ["no_checking", 33, "prior_payments_delayed", "radio_tv", 3713, "500_to_1000", "4_to_7", 4, "female", "none", 3, "savings_insurance", 29, "bank", "own", 1, "unskilled", 1, "none", "yes"],
            ["0_to_200", 24, "outstanding_credit", "appliances", 4162, "500_to_1000", "less_1", 3, "male", "none", 4, "car_other", 38, "bank", "own", 1, "unskilled", 1, "yes", "yes"],
            ["no_checking", 18, "prior_payments_delayed", "vacation", 3078, "500_to_1000", "4_to_7", 3, "female", "none", 4, "savings_insurance", 42, "bank", "own", 1, "unskilled", 1, "yes", "yes"],
            ["no_checking", 40, "prior_payments_delayed", "vacation", 7882, "unknown", "greater_7", 4, "male", "co-applicant", 4, "car_other", 49, "none", "own", 2, "skilled", 2, "yes", "no"],
            ["0_to_200", 10, "credits_paid_to_date", "car_used", 266, "less_100", "less_1", 2, "female", "none", 1, "real_estate", 27, "none", "own", 1, "management_self-employed", 1, "none", "yes"],
            ["no_checking", 45, "prior_payments_delayed", "radio_tv", 5440, "500_to_1000", "1_to_4", 4, "male", "none", 4, "car_other", 38, "none", "own", 1, "skilled", 1, "yes", "yes"],
            ["0_to_200", 9, "prior_payments_delayed", "car_used", 3558, "less_100", "unemployed", 2, "male", "none", 1, "car_other", 31, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 33, "prior_payments_delayed", "radio_tv", 7362, "100_to_500", "4_to_7", 4, "male", "none", 5, "savings_insurance", 45, "none", "own", 2, "management_self-employed", 1, "none", "yes"],
            ["less_0", 6, "credits_paid_to_date", "car_new", 250, "less_100", "1_to_4", 3, "female", "none", 2, "savings_insurance", 36, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 29, "outstanding_credit", "radio_tv", 9276, "500_to_1000", "1_to_4", 4, "male", "co-applicant", 4, "unknown", 55, "none", "free", 3, "management_self-employed", 1, "yes", "yes"],
            ["0_to_200", 4, "all_credits_paid_back", "car_new", 268, "less_100", "1_to_4", 2, "female", "none", 3, "real_estate", 44, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["less_0", 16, "credits_paid_to_date", "car_new", 1784, "500_to_1000", "unemployed", 1, "male", "none", 3, "savings_insurance", 27, "bank", "rent", 1, "unemployed", 1, "none", "yes"],
            ["0_to_200", 19, "credits_paid_to_date", "car_used", 1597, "500_to_1000", "4_to_7", 2, "female", "co-applicant", 3, "car_other", 38, "stores", "own", 1, "unskilled", 1, "none", "yes"],
            ["less_0", 10, "credits_paid_to_date", "radio_tv", 2801, "less_100", "1_to_4", 1, "female", "none", 2, "car_other", 35, "stores", "own", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 26, "prior_payments_delayed", "car_used", 3377, "100_to_500", "1_to_4", 3, "female", "none", 3, "car_other", 32, "none", "own", 1, "skilled", 1, "yes", "yes"],
            ["0_to_200", 17, "credits_paid_to_date", "car_used", 1870, "less_100", "1_to_4", 2, "male", "none", 2, "savings_insurance", 35, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["less_0", 21, "prior_payments_delayed", "car_used", 5055, "less_100", "greater_7", 4, "male", "none", 4, "unknown", 36, "none", "own", 1, "skilled", 2, "yes", "yes"],
            ["no_checking", 27, "outstanding_credit", "furniture", 4897, "500_to_1000", "4_to_7", 4, "male", "none", 5, "car_other", 45, "none", "own", 2, "management_self-employed", 1, "none", "yes"],
            ["no_checking", 42, "prior_payments_delayed", "repairs", 6016, "500_to_1000", "4_to_7", 5, "male", "co-applicant", 4, "car_other", 54, "none", "free", 2, "management_self-employed", 2, "yes", "yes"],
            ["less_0", 12, "all_credits_paid_back", "car_used", 250, "less_100", "1_to_4", 2, "female", "none", 1, "real_estate", 32, "stores", "own", 1, "skilled", 1, "none", "yes"],
            ["greater_200", 9, "credits_paid_to_date", "car_new", 250, "less_100", "less_1", 2, "female", "none", 3, "savings_insurance", 37, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["less_0", 11, "credits_paid_to_date", "furniture", 1732, "less_100", "less_1", 2, "male", "none", 1, "car_other", 32, "stores", "own", 1, "skilled", 1, "none", "yes"],
            ["0_to_200", 4, "prior_payments_delayed", "car_new", 250, "500_to_1000", "4_to_7", 3, "male", "none", 2, "car_other", 43, "stores", "own", 2, "skilled", 1, "yes", "yes"],
            ["less_0", 12, "all_credits_paid_back", "furniture", 1892, "100_to_500", "less_1", 2, "female", "none", 3, "savings_insurance", 31, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["less_0", 13, "all_credits_paid_back", "furniture", 250, "500_to_1000", "4_to_7", 3, "male", "none", 1, "car_other", 36, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["0_to_200", 24, "prior_payments_delayed", "radio_tv", 3395, "100_to_500", "4_to_7", 4, "male", "none", 2, "savings_insurance", 30, "none", "own", 1, "skilled", 2, "yes", "yes"],
            ["no_checking", 18, "outstanding_credit", "business", 8893, "100_to_500", "greater_7", 5, "male", "none", 5, "unknown", 36, "none", "free", 3, "management_self-employed", 1, "yes", "yes"],
            ["no_checking", 27, "prior_payments_delayed", "appliances", 5407, "100_to_500", "4_to_7", 4, "male", "none", 4, "car_other", 41, "stores", "own", 1, "unskilled", 1, "none", "yes"],
            ["0_to_200", 4, "all_credits_paid_back", "car_new", 250, "less_100", "less_1", 2, "male", "none", 2, "real_estate", 19, "none", "rent", 1, "management_self-employed", 1, "none", "yes"],
            ["no_checking", 50, "outstanding_credit", "repairs", 6822, "unknown", "greater_7", 6, "male", "guarantor", 5, "unknown", 57, "none", "free", 3, "skilled", 2, "yes", "yes"],
            ["no_checking", 27, "outstanding_credit", "radio_tv", 2560, "500_to_1000", "4_to_7", 4, "male", "none", 4, "savings_insurance", 36, "stores", "own", 1, "unskilled", 1, "yes", "yes"],
            ["greater_200", 19, "credits_paid_to_date", "vacation", 5214, "less_100", "less_1", 3, "male", "co-applicant", 4, "savings_insurance", 33, "none", "free", 2, "management_self-employed", 1, "none", "no"],
            ["no_checking", 22, "prior_payments_delayed", "radio_tv", 3003, "100_to_500", "greater_7", 4, "male", "none", 4, "real_estate", 33, "none", "own", 2, "skilled", 1, "yes", "yes"],
            ["no_checking", 56, "outstanding_credit", "appliances", 9151, "unknown", "greater_7", 6, "male", "co-applicant", 4, "car_other", 48, "none", "own", 2, "skilled", 2, "none", "no"],
            ["no_checking", 34, "outstanding_credit", "furniture", 4476, "500_to_1000", "greater_7", 4, "male", "none", 3, "savings_insurance", 32, "stores", "own", 1, "unskilled", 1, "none", "yes"],
            ["less_0", 18, "credits_paid_to_date", "vacation", 1588, "500_to_1000", "4_to_7", 3, "male", "none", 3, "car_other", 30, "bank", "own", 1, "unemployed", 1, "yes", "yes"],
            ["no_checking", 21, "prior_payments_delayed", "car_used", 4049, "100_to_500", "greater_7", 4, "male", "none", 4, "savings_insurance", 28, "none", "own", 1, "skilled", 2, "yes", "yes"],
            ["less_0", 24, "credits_paid_to_date", "car_new", 3464, "less_100", "4_to_7", 3, "male", "guarantor", 3, "savings_insurance", 37, "none", "own", 1, "skilled", 1, "yes", "yes"],
            ["greater_200", 4, "credits_paid_to_date", "car_new", 250, "500_to_1000", "1_to_4", 3, "female", "none", 3, "real_estate", 22, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 26, "outstanding_credit", "furniture", 7394, "unknown", "greater_7", 4, "male", "co-applicant", 5, "car_other", 50, "none", "own", 3, "skilled", 2, "yes", "no"],
            ["0_to_200", 24, "outstanding_credit", "appliances", 1056, "100_to_500", "greater_7", 4, "male", "none", 2, "savings_insurance", 29, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 37, "prior_payments_delayed", "radio_tv", 5153, "500_to_1000", "4_to_7", 4, "male", "none", 5, "car_other", 49, "none", "own", 2, "management_self-employed", 1, "yes", "yes"],
            ["less_0", 10, "all_credits_paid_back", "car_new", 1771, "less_100", "1_to_4", 2, "female", "none", 3, "savings_insurance", 39, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["0_to_200", 22, "outstanding_credit", "radio_tv", 5876, "greater_1000", "greater_7", 4, "male", "none", 4, "unknown", 45, "none", "own", 2, "skilled", 1, "yes", "yes"],
            ["less_0", 4, "all_credits_paid_back", "car_new", 250, "less_100", "unemployed", 1, "female", "none", 1, "real_estate", 19, "none", "rent", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 30, "prior_payments_delayed", "appliances", 5614, "unknown", "greater_7", 5, "male", "none", 5, "car_other", 46, "none", "free", 2, "skilled", 2, "yes", "yes"],
            ["less_0", 4, "credits_paid_to_date", "car_new", 839, "less_100", "unemployed", 1, "female", "none", 1, "savings_insurance", 33, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["greater_200", 22, "outstanding_credit", "appliances", 4013, "greater_1000", "4_to_7", 3, "male", "none", 2, "car_other", 43, "none", "free", 2, "skilled", 2, "yes", "yes"],
            ["0_to_200", 13, "credits_paid_to_date", "car_new", 907, "less_100", "less_1", 2, "male", "none", 3, "savings_insurance", 28, "stores", "own", 1, "skilled", 1, "none", "yes"],
            ["less_0", 15, "credits_paid_to_date", "car_used", 250, "500_to_1000", "1_to_4", 2, "male", "none", 3, "savings_insurance", 26, "stores", "rent", 1, "skilled", 1, "none", "yes"],
            ["greater_200", 9, "credits_paid_to_date", "furniture", 297, "less_100", "less_1", 3, "female", "none", 2, "real_estate", 25, "none", "rent", 1, "management_self-employed", 1, "none", "yes"],
            ["0_to_200", 23, "credits_paid_to_date", "car_new", 3085, "less_100", "less_1", 3, "male", "none", 2, "savings_insurance", 37, "none", "rent", 1, "skilled", 1, "none", "yes"],
            ["0_to_200", 18, "credits_paid_to_date", "furniture", 2572, "500_to_1000", "4_to_7", 3, "female", "none", 3, "real_estate", 29, "none", "rent", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 42, "prior_payments_delayed", "vacation", 6531, "500_to_1000", "1_to_4", 4, "male", "co-applicant", 4, "unknown", 49, "none", "free", 2, "skilled", 1, "yes", "yes"],
            ["0_to_200", 10, "prior_payments_delayed", "furniture", 250, "100_to_500", "less_1", 2, "male", "none", 3, "savings_insurance", 29, "stores", "rent", 1, "skilled", 2, "none", "yes"],
            ["0_to_200", 25, "outstanding_credit", "car_used", 2585, "500_to_1000", "greater_7", 4, "male", "none", 3, "unknown", 39, "none", "own", 2, "skilled", 1, "yes", "yes"],
            ["less_0", 16, "prior_payments_delayed", "radio_tv", 3975, "500_to_1000", "1_to_4", 3, "male", "none", 2, "car_other", 35, "bank", "rent", 2, "unemployed", 1, "none", "yes"],
            ["less_0", 41, "prior_payments_delayed", "furniture", 4764, "greater_1000", "less_1", 3, "female", "none", 3, "unknown", 30, "bank", "own", 2, "unskilled", 1, "none", "yes"],
            ["no_checking", 19, "credits_paid_to_date", "car_new", 5375, "100_to_500", "less_1", 4, "female", "co-applicant", 3, "real_estate", 51, "none", "own", 1, "management_self-employed", 2, "yes", "no"],
            ["no_checking", 35, "prior_payments_delayed", "appliances", 5478, "500_to_1000", "less_1", 4, "male", "none", 3, "car_other", 47, "stores", "own", 2, "unskilled", 1, "yes", "yes"],
            ["0_to_200", 16, "credits_paid_to_date", "vacation", 986, "less_100", "less_1", 3, "male", "none", 2, "real_estate", 19, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["0_to_200", 16, "credits_paid_to_date", "car_new", 1230, "500_to_1000", "1_to_4", 2, "female", "none", 3, "savings_insurance", 31, "stores", "own", 1, "skilled", 2, "yes", "yes"],
            ["less_0", 4, "all_credits_paid_back", "vacation", 1026, "less_100", "1_to_4", 1, "female", "none", 2, "real_estate", 19, "none", "rent", 1, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "car_used", 3812, "unknown", "4_to_7", 4, "male", "none", 3, "unknown", 45, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 16, "credits_paid_to_date", "furniture", 1530, "100_to_500", "1_to_4", 3, "male", "none", 4, "real_estate", 30, "none", "own", 1, "management_self-employed", 1, "none", "yes"],
            ["no_checking", 36, "credits_paid_to_date", "furniture", 5024, "100_to_500", "greater_7", 4, "male", "none", 4, "car_other", 47, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 23, "prior_payments_delayed", "radio_tv", 5214, "500_to_1000", "4_to_7", 3, "male", "none", 4, "savings_insurance", 50, "none", "own", 1, "management_self-employed", 1, "none", "yes"],
            ["0_to_200", 22, "credits_paid_to_date", "repairs", 1883, "less_100", "1_to_4", 3, "male", "none", 2, "savings_insurance", 26, "bank", "rent", 1, "unskilled", 1, "none", "yes"],
            ["no_checking", 32, "prior_payments_delayed", "appliances", 2160, "500_to_1000", "4_to_7", 3, "male", "none", 2, "car_other", 37, "none", "own", 1, "skilled", 1, "none", "yes"],
            ["less_0", 9, "prior_payments_delayed", "vacation", 7128, "500_to_1000", "1_to_4", 3, "male", "none", 3, "car_other", 32, "none", "own", 2, "management_self-employed", 1, "yes", "yes"],
            ["less_0", 4, "all_credits_paid_back", "repairs", 250, "less_100", "less_1", 1, "male", "none", 1, "real_estate", 19, "bank", "rent", 1, "unemployed", 1, "none", "yes"],
            ["less_0", 45, "credits_paid_to_date", "other", 5036, "less_100", "less_1", 3, "male", "none", 3, "savings_insurance", 29, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["less_0", 12, "credits_paid_to_date", "furniture", 250, "500_to_1000", "1_to_4", 1, "male", "none", 1, "real_estate", 23, "stores", "rent", 1, "unskilled", 1, "none", "yes"]
        ],
        "fields": ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings", "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone", "ForeignWorker"]
        }]
}
response = client.deployments.score(deployment_uid, scoring_payload)
predictions = response["predictions"]

In [ ]:
predictions

### Configure Watson Openscale
https://aiopenscale.cloud.ibm.com/aiopenscale?features=v4

In [ ]:
# Payload logging
from ibm_watson_openscale import APIClient as WOS_CLIENT
from ibm_watson_openscale.utils import get_instance_guid
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

WOS_URL = "https://api.aiopenscale.cloud.ibm.com"
IAM_URL = "https://iam.ng.bluemix.net/oidc/token"
authenticator = IAMAuthenticator(
                    apikey=CLOUD_API_KEY,
                    url=IAM_URL
                )
WOS_GUID = get_instance_guid(authenticator)
print("Watson Openscale Instance ID: {}".format(WOS_GUID))
wos_client = WOS_CLIENT(authenticator=authenticator, service_url=WOS_URL)
wos_client.version

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_name = subscription.entity.asset.name
    if sub_model_name == MODEL_NAME:
        subscription_uid = subscription.metadata.id
subscription_uid   

In [ ]:
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
import uuid
# Get the data set id
data_sets = wos_client.data_sets.list(target_target_id=subscription_uid, type="payload_logging").result.data_sets
data_set_id = data_sets[0].metadata.id
print("Payload data_set_id: ", data_set_id)
scoring_id="wos_test_id_" + str(uuid.uuid4())
pl_records = PayloadRecord(scoring_id=scoring_id, request=scoring_payload["input_data"][0], response=predictions[0])
store_records_response = wos_client.data_sets.store_records(request_body=[pl_records], data_set_id=data_set_id)
print(store_records_response.result.to_dict())